In [50]:
# import pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import random
random.seed(0)
seed = 0

In [71]:
"""
filename = 'data/train_conll_spanglish.csv'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torchtext

def label2int(label):
    if label=='positive':
        return 1
    elif label=='negative':
        return -1
    else:
        return 0

text_field = torchtext.data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True)       # to turn each character into an integer index
label_field = torchtext.data.Field(sequential=False,    # not a sequence
                                   use_vocab=False,     # don't need to track vocabulary
                                   is_target=True,      
                                   batch_first=True,
                                   preprocessing=lambda x: label2int(x)) # convert text to 0 and 1

fields = [('id', None),('text', text_field), ('label', label_field)]
dataset = torchtext.data.TabularDataset(filename, # name of the file
                                        "tsv",               # fields are separated by a tab
                                        fields)
"""

In [72]:
for i in range(0,10):
    print(dataset[i].text, "---", dataset[i].label)

So that means tomorrow cruda segura lol --- 1
Tonight peda segura --- 0
Eres tan mala vieja bruja interesada#jamming --- -1
Yo kiero Pretzels lol --- 0
Fuck that ni ke el me vaya a mantener toda la vida lol --- -1
I always tell my dad ke me kiero kasar con una vieja rika and me regaña telling me ke no sea interesada ha --- -1
Ke me compre un carrito pa irme con mis friends and party lol --- 0
Why can I just find a rich bitch ke me mantenga y ya ha --- 0
Since I started working ya ni disfruto la vida lol --- -1
My dad me regano cuzs I was telling that to my brother and lo andaba molestando lol --- -1


In [73]:
train, val, test = dataset.split(split_ratio=[0.8,0.1,0.1])

In [74]:
text_field.build_vocab(dataset)
text_field.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x128d00160>>,
            {'<unk>': 0,
             '<pad>': 1,
             ' ': 2,
             'e': 3,
             'a': 4,
             'o': 5,
             't': 6,
             's': 7,
             'n': 8,
             'i': 9,
             'r': 10,
             'l': 11,
             'c': 12,
             'd': 13,
             'u': 14,
             'm': 15,
             'h': 16,
             'p': 17,
             'y': 18,
             '.': 19,
             '/': 20,
             'g': 21,
             'b': 22,
             'v': 23,
             ':': 24,
             '@': 25,
             '!': 26,
             'f': 27,
             'A': 28,
             'j': 29,
             'E': 30,
             'T': 31,
             'R': 32,
             'q': 33,
             'k': 34,
             '#': 35,
             'w': 36,
             'M': 37,
             'S': 38,
             'O': 39,
             'L': 40

In [75]:
text_field.vocab.itos

['<unk>',
 '<pad>',
 ' ',
 'e',
 'a',
 'o',
 't',
 's',
 'n',
 'i',
 'r',
 'l',
 'c',
 'd',
 'u',
 'm',
 'h',
 'p',
 'y',
 '.',
 '/',
 'g',
 'b',
 'v',
 ':',
 '@',
 '!',
 'f',
 'A',
 'j',
 'E',
 'T',
 'R',
 'q',
 'k',
 '#',
 'w',
 'M',
 'S',
 'O',
 'L',
 'I',
 'N',
 'P',
 'C',
 'D',
 'z',
 ',',
 'G',
 'B',
 '1',
 'Y',
 '0',
 'J',
 'H',
 'U',
 'F',
 'x',
 'V',
 '2',
 '_',
 'í',
 'W',
 '"',
 '3',
 'K',
 '7',
 '?',
 'Z',
 '6',
 'Q',
 '4',
 '5',
 '8',
 '9',
 'á',
 "'",
 'ñ',
 '…',
 'é',
 'X',
 'ó',
 '-',
 '😂',
 ')',
 '(',
 '️',
 '|',
 '❤',
 '😍',
 'ú',
 '’',
 '“',
 '”',
 '^',
 '&',
 '¿',
 '😭',
 '*',
 '>',
 '$',
 ';',
 '😁',
 '¡',
 '👌',
 '・',
 '🔥',
 '🏼',
 '😱',
 '☺',
 '🚨',
 '🎉',
 '👏',
 'Í',
 '😘',
 'É',
 '😝',
 '😎',
 '🏻',
 '🏽',
 '🙌',
 '💕',
 '😊',
 '~',
 '♥',
 '•',
 'Á',
 'Ñ',
 '<',
 '🎵',
 '😩',
 '💀',
 '+',
 '😜',
 '👍',
 '😋',
 '😒',
 '✌',
 '😏',
 '😡',
 '%',
 'Ó',
 '👗',
 '💃',
 '💋',
 '💪',
 '=',
 '☀',
 '💁',
 '😔',
 '🎶',
 '😉',
 '😳',
 '😌',
 '🤔',
 '💯',
 '🍻',
 '👇',
 '😈',
 '🙄',
 '🙋',
 '👠',
 '🌸',
 '😅',
 '😫',
 '

In [76]:
train_iter = torchtext.data.BucketIterator(train,
                                           batch_size=32,
                                           sort_key=lambda x: len(x.text), # to minimize padding
                                           sort_within_batch=True,        # sort within each batch
                                           repeat=True, # repeat the iterator for multiple epochs
                                           device=device)
val_iter = torchtext.data.BucketIterator(val,
                                         batch_size=32,
                                         sort_key=lambda x: len(x.text), # to minimize padding
                                         sort_within_batch=True,        # sort within each batch
                                         repeat=True, # repeat the iterator for multiple epochs
                                         device=device)
test_iter = torchtext.data.BucketIterator(test,
                                          batch_size=32,
                                          sort_key=lambda x: len(x.text), # to minimize padding
                                          sort_within_batch=True,        # sort within each batch
                                          repeat=True, # repeat the iterator for multiple epochs
                                          device=device)

In [77]:
for i, batch in enumerate(train_iter):
    if i >= 2:
        break
    print(batch.text)
#     print(batch.text[0].shape)
    print(batch.label)

(tensor([[43,  4, 10,  ..., 37, 30, 72],
        [32, 31, 25,  ..., 44, 27, 58],
        [32, 31, 25,  ..., 20, 78,  1],
        ...,
        [39, 37, 48,  ...,  3,  1,  1],
        [44, 14,  6,  ...,  6,  1,  1],
        [25, 44,  9,  ..., 11,  1,  1]]), tensor([132, 132, 131, 131, 131, 131, 131, 131, 131, 131, 131, 131, 131, 131,
        131, 131, 131, 131, 131, 131, 131, 131, 131, 130, 130, 130, 130, 130,
        130, 130, 130, 130]))
tensor([-1,  1,  0,  1,  1,  1,  0,  0,  0,  1,  1,  0, -1,  1,  0,  0, -1,  1,
        -1,  1,  0,  0,  1,  0, -1,  1,  0,  1,  0,  1,  1,  1])
(tensor([[ 35,  32,   3,  ...,   4,   4,  40],
        [ 25,  22,  10,  ...,  24,  84, 124],
        [ 54,   3,  18,  ...,  29,  74,  80],
        ...,
        [ 44,   5,   8,  ...,   6,  11,   1],
        [ 44,  10,   3,  ...,  23,  14,   1],
        [ 54,   3,  12,  ...,  31,  45,   1]]), tensor([122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 122, 121, 121, 121,
        121, 121, 121, 121, 121, 121, 121, 

In [101]:
"""
Another version of preprocessing data
"""
import sklearn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import os

INPUT_PATH = "data/train_conll_spanglish.csv"
MAX_TWEET = 280

char_to_ind = {}
ind_to_char = {}

char_to_ind.update({"UNK":0})
ind_to_char.update({0:"UNK"})

count = 1

with open(INPUT_PATH, 'r') as f:
    for line in f:
        for char in line.split('\t')[1]:
            if char.lower() not in char_to_ind:
                char_to_ind.update({char.lower():count})
                ind_to_char.update({count:char.lower()})
                count += 1

#print(char_to_ind)
#print(ind_to_char)

n_letters = len(char_to_ind)

In [102]:
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][char_to_ind[letter]] = 1
    return tensor

def lineToTensor(line):
    tensor = torch.zeros(len(line), n_letters)
    for li, letter in enumerate(line):
        tensor[li][char_to_ind[letter]] = 1
    return tensor

def batchToTensor(batch):
    tensor = torch.zeros(len(batch),MAX_TWEET,n_letters)
    for sentence, line in enumerate(batch):
        for li, letter in enumerate(line):
            tensor[sentence][li][char_to_ind[letter.lower()]] = 1
    return tensor


#print(letterToTensor('o'))
print(lineToTensor('hello how are tou').shape)
print(batchToTensor(['hello friend', 'linear svm is better']))
print(batchToTensor(['hello friend', 'linear svm is better']).shape)

torch.Size([17, 554])
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
torch.Size([2, 280, 554])


In [103]:
trainpath = os.path.join("data", "train_conll_spanglish.csv")
train = pd.read_csv(trainpath, sep='\\t', names=["ID","SENTENCE","LABEL"])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [106]:
print(train['SENTENCE'][0].lower())
train_char_features = batchToTensor(train['SENTENCE'])

so that means tomorrow cruda segura lol


In [107]:
print(train_char_features.shape)


torch.Size([15000, 280, 554])


In [97]:
class TextCNN(nn.Module):
    """
    TextCNN implementation based on
    https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb
    """
    def __init__(self, vocab_size, embed_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # first convolutional layer (three layers)
        self.conv_0 = nn.ModuleList([
                nn.Conv2d(in_channels = 1,
                          out_channels = n_filters,
                          kernel_size = (fs, embed_dim))
                for fs in filter_sizes
        ])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
    
    def forward(self, text):
        # text = (tensor of input, tensor of input length)
        print(text[0].shape)
        # convert input to embeddings
        in_data = text[0]
        # in_data = [batch_size, sentence_length]
        embedded = self.embedding(in_data)
        # embedded = [batch_size, sentence_length, embedding_dimension]
        embedded = embedded.unsqueeze(1)
        # embedded = [batch_size, 1, sentence_length, embedding_dimension]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.conv_0]
        # conved_n = [batch_size, n_filters, sentence_length - filter_size[n] - 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        # pooled_n = [batch_size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim=1))
        # cat = [batch_size, n_filters * len(filter_sizes)]
        return self.fc(cat)    
        

In [111]:
# train_char_features = [num_of_tweets, max. length of each tweet, embedding_size]
num_features = list(train_char_features.shape)
print(num_features)
max_tweet_length = num_features[1]
embedding_dim = num_features[2] # vocab_size
n_filters = 3 # number of filters
filter_sizes = [3, 4, 5] # like character 3-gram, 4-gram, 5-gram
output_dim = 3
dropout = 0.5

model = TextCNN(input_dim, embedding_dim, n_filters, filter_sizes, output_dim, dropout)

[15000, 280, 554]


In [112]:
# checking the parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 22,753 trainable parameters


In [113]:
# training
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [92]:
def get_accuracy(model, data_iter):
    correct, total = 0, 0
    for i, batch in enumerate(data_iter):
        output = model(batch.text[0]) # You may need to modify this, depending on your model setup
        print(output)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(batch.label.view_as(pred)).sum().item()
        total += batch.text[1].shape[0]
    return correct / total

In [93]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = get_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [94]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = get_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [95]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [99]:
N_EPOCHS = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

torch.Size([32, 79])


RuntimeError: index out of range: Tried to access index 25 out of table with 4 rows. at ../aten/src/TH/generic/THTensorEvenMoreMath.cpp:237